In [1]:
# default_exp survey

# survey module

> does stuff, tbd.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
import openai, pandas as pd, numpy as np, datetime, json
from OpenAISurveyWrapper import wrapper

In [171]:
#export
class survey(wrapper.wrapper):
    """
    wrapper on the API to load in survey definitions
    a format for displaying
    
    self.myFormat: a function to format a prompt
    """
    labeled = pd.DataFrame()
    raw = pd.DataFrame()
    examples = pd.DataFrame()
    definitions = {}
    myFormat = """A post is about {} if it {} Consider the following post:\npost: "{}"\nq: Was the post about {} (yes or no)?\na:""".format
    kwargs = {
        "engine":"davinci",
        "temperature":0,
        "max_tokens":10,
        "stop":"\n",
    }
    
    def setFormat(self, func):
        """
        set prompt format to newFormat
        """
        self.myFormat = func
        
    def formatQuery(self, *args):
        return (self.myFormat(*args))
            
    def addDefinition(self, label:str, definition:str):
        """
        set definition of label to definition)
        """
        self.definitions[label] = definition
        
    def removeDefinition(self, label:str):
        """
        set definition of label to definition)
        """
        del self.definitions[label]

    def loadExamplesFromDataFrame(self, examples:dict):
        """
        takes a 2 column dataframe; first column should be "text", second "label"
        """
        if len(examples.columns)!=2:
            raise Exception("need 2 columns, first should be data second label")
        self.examples = examples
        self.examples.columns=["data", "label"]
        
    def generateResponses(self, queries, prompt, verbose=False):
        """
        run a prompt against queries.
        prompt should be formated as "str {} str" to be able to use format
        to stick each query in
        """
        print("before")
        
        results = []
        for query in queries:
            r = self.query(prompt=prompt.format(query[:1500]), **self.kwargs)["choices"][0]["text"]
            if verbose:
                print(r)
            results.append(r)
        
        print("after")
        print(results)
        return results
        
    def tryLabels(self, n=20, overwrite=True, inplace=True, fewShot = 0, labels=[], verbose=False):
        """
        for each label in labels, add to the 
        """
        if len(labels) == 0:
            labels = [x for x in self.definitions]
        
        if n > 0:
            targets = self.examples.sample(n)
        else:
            targets = self.examples.copy()
        
        
        all_responses = {}
        for label in labels:
                prompt = self.formatQuery(label, self.definitions[label],"{}", label)
                print(prompt)
                responses = self.generateResponses(targets["data"].to_list(), prompt, verbose=verbose)
                
                with open("{}/{}".format(self.outdir, datetime.datetime.now().strftime("%Y%m%d%H%m%S") + ".json"), "w") as fh:
                    json.dump({"prompt":prompt, "data":{"queries":targets["data"].to_list(), "preds":responses}}, fh, indent=4)

                all_responses[label]=responses
                
                targets[label] = responses
                self.labeled = targets
        self.labeled = targets

        

In [159]:
term = "Federal Budget"
definition="refers to mentions of how the federal government budgets and spends its money."
post = "the irish are heavy drinkers"

In [160]:
tmp = survey()

In [161]:
ls ../gpt3

CommaComparisonChart.png        facebook_examples.csv
DollarCommaNocomma.png          key.json
FullCommaNoCommaSpace.png       labeledexamples.csv
Is Polarization a Myth.pdf      no commas.json
TruncatedCommaNoCommaSpace.png  pypirc
combinedata.forml.xlsx          spaces.json
commas.json


In [162]:
tmp.add_key(json.load(open("../gpt3/key.json", "r"))["key"])

In [163]:
tmp.addDefinition(term, definition)

In [164]:
df = pd.read_excel("../combinedata.forml.xlsx")

In [166]:
tmp.loadExamplesFromDataFrame(df[["text", "classification"]])

In [167]:
tmp.tryLabels()

A post is about Federal Budget if it refers to mentions of how the federal government budgets and spends its money. Consider the following post:
post: "{}"
q: Does this post primarily about Federal Budget (yes or no)?
a:
before
 Yes
 yes
 yes
 No
 No
 No
 No
 Yes
 Yes
 No
 yes
 No
 No
 Yes
 yes
 No
 yes
 Yes
 yes
 yes
after
[' Yes', ' yes', ' yes', ' No', ' No', ' No', ' No', ' Yes', ' Yes', ' No', ' yes', ' No', ' No', ' Yes', ' yes', ' No', ' yes', ' Yes', ' yes', ' yes']


In [168]:
tmp.labeled

,data,label,Federal Budget
834,Why is it that we keep having mass shooting af...,gun control,Yes
2110,Twana,none,yes
2516,The new people getting their info correct and ...,trump as issue,yes
98,I posted this a couple of years ago: The Monke...,drugs,No
2452,So a “white nationalist” group that’s never be...,racial equality,No
2450,As an avid RV'er I will be driving right by Ki...,racial equality,No
597,CRIMES OF THE PUTIN REGIME AGAINST HUMANITY B...,foreign affairs,No
1255,I think it's sooo good to see him that he was ...,none,Yes
1940,"... who's been watching ""Magic Mike"" on my Net...",none,Yes
2223,#ITfavorites #entry cc cream,none,No


In [146]:
tmp.definitions

{'Federal Budget': 'refers to mentions of how the federal government budgets and spends itsmoney. Do not label state funds or expenditures as federal. Concepts associated with the federal budget are commonly the federal deficit, national debt and military spending.'}

In [147]:
tmp.formatQuery(term, definition,term, post, term)

'A post is about Federal Budget if it refers to mentions of how the federal government budgets and spends itsmoney. Do not label state funds or expenditures as federal. Concepts associated with the federal budget are commonly the federal deficit, national debt and military spending. Consider the following post:\npost: "Federal Budget"\nq: Does this post discuss the irish are heavy drinkers (yes or no)?\na:'

In [143]:
#hide


In [144]:
df

,text,type,classification
0,We have problems with immigrations. The child...,survey,immigration
1,Immigrations.,survey,immigration
2,The deficit!,survey,federal budget
3,Deficit,survey,federal budget
4,The deficit,survey,federal budget
...,...,...,...
2615,"So the refs get rings, too, right???",post,none
2616,Lori Colello,post,none
2617,"14 The LORD shall fight for you, and ye shall...",post,none
2618,#ITfavorites #entry cc cream,post,none
